In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import numpy as np
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.tools import mpl_to_plotly
from typing import Callable

# Inicializar la aplicación Dash con Bootstrap para los estilos
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Definir el diseño de la aplicación usando componentes de Dash Bootstrap
app.layout = dbc.Container(
    [
        # Mensaje inicial con introducción y las instrucciones
        dbc.Alert(
            [
                html.H4("¡Bienvenido a la Interfaz de Control de Funciones de Población!", className="alert-heading"),
                html.P("Esta aplicación te permite visualizar y controlar un modelo de población basado en el sistema iterativo del mapa logístico."),
                html.Hr(),
                html.P("Instrucciones de Uso:"),
                html.Ul([
                    html.Li("Use el deslizador 'f' para ajustar la tasa de fertilidad."),
                    html.Li("Use el deslizador 'P0' para ajustar la población inicial."),
                    html.Li("El primer gráfico muestra el diagrama de cobweb para los valores dados de 'f' y 'P0'."),
                    html.Li("El segundo gráfico muestra la evolución de la población con el tiempo."),
                    html.Li("El tercer gráfico muestra el diagrama de bifurcación."),
                ]),
            ],
            color="info",  # Color del mensaje
            dismissable=True,  # Permite cerrar el mensaje
            style={'margin-bottom': '20px'}  # Espaciado inferior
        ),
        # Encabezado de la aplicación
        html.H1("Interfaz de Control de Funciones de Población", style={'textAlign': 'center', 'margin-bottom': '30px'}),

        # Fila que contiene los controles y los gráficos
        dbc.Row(
            [
                # Columna para los deslizadores y botones
                dbc.Col(
                    [
                        # Deslizador para la tasa de fertilidad (f)
                        html.Label("f (tasa de fertilidad)"),
                        html.Div([
                            html.Span("0", style={'margin-right': '10px'}),
                            html.Div(id='slider-f-output', style={'margin-bottom': '10px'}),
                            html.Span("4", style={'margin-left': '10px'}),
                        ], style={'display': 'flex', 'justify-content': 'space-between'}),
                        dcc.Slider(
                            id="slider-f",
                            min=0,
                            max=4,
                            step=0.01,
                            value=2,
                            updatemode='drag',
                        ),
                        html.Br(),

                        # Deslizador para la población inicial (P0)
                        html.Label("P0 (población inicial)"),
                        html.Div([
                            html.Span("0", style={'margin-right': '10px'}),
                            html.Div(id='slider-p0-output', style={'margin-bottom': '10px'}),
                            html.Span("1", style={'margin-left': '10px'}),
                        ], style={'display': 'flex', 'justify-content': 'space-between'}),
                        dcc.Slider(
                            id="slider-p0",
                            min=0,
                            max=1,
                            step=0.01,
                            value=0.5,
                            updatemode='drag',
                        ),
                        html.Br(),

                        # Botón para abrir el modal de instrucciones
                        dbc.Button("INSTRUCCIONES", id="open-instructions", color="primary", className="w-100 mb-2"),

                        # Modal que contiene las instrucciones de uso
                        dbc.Modal(
                            [
                                dbc.ModalHeader("Instrucciones de Uso"),
                                dbc.ModalBody(
                                    html.Div([
                                        html.P("1. Use el deslizador 'f' para ajustar la tasa de fertilidad."),
                                        html.P("2. Use el deslizador 'P0' para ajustar la población inicial."),
                                        html.P("3. El primer gráfico muestra el diagrama de cobweb para los valores dados de 'f' y 'P0'."),
                                        html.P("4. El segundo gráfico muestra la evolución de la población con el tiempo."),
                                        html.P("5. El tercer gráfico muestra el diagrama de bifurcación."),
                                    ])
                                ),
                                dbc.ModalFooter(
                                    dbc.Button("Cerrar", id="close-instructions", className="ml-auto")
                                ),
                            ],
                            id="instructions-modal",
                            is_open=False,
                        ),
                        html.Br(),

                        # Botón para abrir el modal de detalles del proyecto
                        dbc.Button("DETALLES", id="open-details", color="primary", className="w-100 mb-2"),

                        # Modal que contiene los detalles del proyecto
                        dbc.Modal(
                            [
                                dbc.ModalHeader("Detalles del Proyecto"),
                                dbc.ModalBody(
                                    html.Div([
                                        html.P("Este proyecto es una aplicación web desarrollada usando Dash y Plotly."),
                                        html.P("La aplicación permite visualizar cómo cambia un modelo de población basado en un sistema iterativo."),
                                        html.P("Los gráficos mostrados incluyen un diagrama de cobweb, la evolución de la población, y un diagrama de bifurcación."),
                                        html.P("Dependiendo del" +
                                               "valor de f, la población puede extinguirse, estabilizarse, o mostrar comportamientos caóticos. Mediante simulaciones en" + 
                                            "Python, se exploran puntos fijos, bifurcaciones y caos para analizar cómo las variaciones en f y P afectan la dinámica poblacional."),
                                    ])
                                ),
                                dbc.ModalFooter(
                                    dbc.Button("Cerrar", id="close-details", className="ml-auto")
                                ),
                            ],
                            id="details-modal",
                            is_open=False,
                        ),
                        html.Br(),

                        # Botón para abrir el modal de información adicional
                        dbc.Button("ACERCA DE", id="open-info", color="primary", className="w-100"),

                        # Modal con información adicional del proyecto
                        dbc.Modal(
                            [
                                dbc.ModalHeader("Información del Proyecto"),
                                dbc.ModalBody(
                                    html.Div([
                                        html.P("ESCUELA POLITÉCNICA NACIONAL"),
                                        html.P("FACULTAD DE INGENIERÍA EN SISTEMAS"),
                                        html.P("Materia: Métodos Numéricos"),
                                        html.P("Proyecto Bimestral"),
                                        html.P("TEMA: Población"),
                                        html.P("Integrantes:"),
                                        html.Ul([
                                            html.Li("BRAVO ORELLANA LEANDRO OMAR"),
                                            html.Li("FLORES CRUZ JUAN ESTEBAN"),
                                            html.Li("JITALA PUMA STALIN LEONIDAS"),
                                            html.Li("ORTIZ CUNDAR BRAYAN PAUL"),
                                        ]),
                                        html.P("Quito, 16 de agosto de 2024"),
                                    ])
                                ),
                                dbc.ModalFooter(
                                    dbc.Button("Cerrar", id="close-info", className="ml-auto")
                                ),
                            ],
                            id="info-modal",
                            is_open=False,
                        ),
                        html.Br(),

                        # Mostrar el valor futuro de la población
                        html.Div(id='future-population-output', style={'margin-top': '50px'}),
                    ],
                    width=2,  # Establecer el ancho de la columna
                ),

                # Columna para el gráfico Cobweb
                dbc.Col(
                    dcc.Graph(id="graph-cobweb"),
                    width=3,
                    style={'padding': '0 5px'}  # Agregar espaciado alrededor del gráfico
                ),

                # Columna para el gráfico de Evolución de la Población
                dbc.Col(
                    dcc.Graph(id="graph-evolution"),
                    width=3,
                    style={'padding': '0 5px'}
                ),

                # Columna para el diagrama de Bifurcación
                dbc.Col(
                    dcc.Graph(id="graph-bifurcation"),
                    width=3,
                    style={'padding': '0 5px'}
                ),
            ],
            align="center",  # Alinear el contenido de la fila al centro
            style={'margin': '0 -10px'}  # Ajustar margen para la fila
        ),
        html.H1("EDO - EULER", style={'textAlign': 'center', 'fontSize': 40}),
        
        #Graficos de abajo
        dbc.Row(
            dbc.Col(
                dcc.Graph(id='matplotlib-plot'),
                style={'display': 'flex', 'justify-content': 'center'}
            )
        ),
        # Componentes de almacenamiento para mantener el estado entre callbacks
        dcc.Store(id='store-f', data={'value': 2}),
        dcc.Store(id='store-p0', data={'value': 0.5}),
    ],
    fluid=True,  # Usar diseño fluido
)

# Función para calcular el modelo de población usando la ecuación del mapa logístico
def population_model(f, P0, num_iterations=30):
    P = np.zeros(num_iterations)  # Inicializar el array de población
    P[0] = P0  # Establecer población inicial
    for n in range(1, num_iterations):
        P[n] = f * P[n-1] * (1 - P[n-1])  # Calcular población para cada iteración
    return P

# Función para calcular el valor futuro de la población
def p_futura(f, P0, num_iterations=30):
    P = 0.5
    if f < 1:
        P = 0
    elif f < 3:
        P = (f - 1) / f
    elif f < 3.5:
        P = (f - 1) / f
    return P  # Retorna el valor de la última iteración

# Función para crear el gráfico Cobweb
def cobweb_plot(f, P0, num_iterations=30):
    P = population_model(f, P0, num_iterations)
    x = np.linspace(0, 1, 200)  # Valores de x para trazar la curva
    y = f * x * (1 - x)  # Valores de y para trazar el mapa logístico

    # Crear las líneas del cobweb para el gráfico
    cobweb_lines = []
    cobweb_lines.append(go.Scatter(
        x=[P0, P0],
        y=[0, P[1]],
        mode='lines',
        line=dict(color='green', width=2),
        name='Cobweb'
    ))

    # Agregar líneas para cada iteración del cobweb
    for i in range(1, num_iterations):
        cobweb_lines.append(go.Scatter(
            x=[P[i-1], P[i]],
            y=[P[i], P[i]],
            mode='lines',
            line=dict(color='green', width=2),
            showlegend=False
        ))
        cobweb_lines.append(go.Scatter(
            x=[P[i], P[i]],
            y=[P[i], f * P[i] * (1 - P[i])],
            mode='lines',
            line=dict(color='green', width=2),
            showlegend=False
        ))

    # Devolver la figura del gráfico Cobweb
    return go.Figure(data=[
        go.Scatter(x=x, y=y, mode='lines', name='f * P * (1 - P)', line=dict(color='blue', width=2)),
        go.Scatter(x=x, y=x, mode='lines', name='y = x', line=dict(color='red', width=2)),
        *cobweb_lines
    ],
    layout=go.Layout(
        title='Gráfico Cobweb',
        xaxis_title='Población',
        yaxis_title='Población Siguiente',
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[0, 1]),
        title_x=0.5,
        title_y=0.95,
        legend=dict(orientation="h", x=0.5, xanchor='center', y=-0.2),
        margin=dict(l=20, r=20, t=40, b=20),
        height=400  # Altura fija
    ))

# Función para crear el gráfico de Evolución de la Población
def population_evolution(f, P0, num_iterations=50):
    P = population_model(f, P0, num_iterations)
    # Devolver la figura del gráfico de Evolución de la Población
    return go.Figure(data=[
        go.Scatter(x=np.arange(num_iterations), y=P, mode='lines', line=dict(color='blue', width=2))
    ],
    layout=go.Layout(
        title='Evolución de la Población',
        xaxis_title='Tiempo',
        yaxis_title='Población',
        yaxis=dict(range=[0, 1]),  # Rango fijo en el eje y
        title_x=0.5,
        title_y=0.95,
        margin=dict(l=20, r=20, t=40, b=20),
        height=400  # Altura fija
    ))

# Función para crear el diagrama de Bifurcación
def bifurcation_diagram(f_min=0, f_max=4, num_points=1000):
    f_values = np.linspace(f_min, f_max, num_points)  # Valores de la constante de fertilidad
    P0 = 0.5  # Población inicial
    num_iterations = 1000
    last_iterations = 20  # Número de iteraciones a trazar
    P_values = []
    for f in f_values:
        P = population_model(f, P0, num_iterations)
        P_values.append(P[-last_iterations:])  # Almacenar las últimas iteraciones
    
    flat_P_values = [p for sublist in P_values for p in sublist]  # Aplanar los valores de la población
    f_repeated = np.repeat(f_values, last_iterations)  # Repetir los valores de fertilidad

    # Devolver la figura del diagrama de Bifurcación
    return go.Figure(data=[
        go.Scatter(x=f_repeated, y=flat_P_values, mode='markers', marker=dict(color='black', size=0.8))
    ],
    layout=go.Layout(
        title='Diagrama de Bifurcación',
        xaxis_title='Constante de Fertilidad (f)',
        yaxis_title='Población (P)',
        yaxis=dict(range=[0, 1]),  # Rango fijo en el eje y
        title_x=0.5,
        title_y=0.95,
        margin=dict(l=20, r=20, t=40, b=20),
        height=400  # Altura fija
    ))

# Callback para actualizar los gráficos y mostrar valores de los deslizadores
@app.callback(
    [Output("graph-cobweb", "figure"), Output("graph-evolution", "figure"), Output("slider-f-output", "children"), Output("slider-p0-output", "children"), Output('future-population-output', 'children')],
    [Input("slider-f", "value"), Input("slider-p0", "value")],
    [State('store-f', 'data'), State('store-p0', 'data')]
)
def update_graphs(f, P0, stored_f, stored_p0):
    if f is None or P0 is None:
        raise PreventUpdate
    
    # Generar las figuras actualizadas basadas en los valores de los deslizadores
    fig_cobweb = cobweb_plot(f, P0)
    fig_evolution = population_evolution(f, P0)
    
    # Calcular el valor futuro de la población
    future_population = p_futura(f, P0)
    
    # Devolver las figuras actualizadas, los valores de los deslizadores, y el valor futuro de la población
    return fig_cobweb, fig_evolution, f"{f:.2f}", f"{P0:.2f}", f"Valor Futuro de la Población: {future_population:.2f}"

# Callback para actualizar el valor almacenado de la tasa de fertilidad
@app.callback(
    Output('store-f', 'data'),
    [Input('slider-f', 'value')]
)
def update_store_f(f):
    return {'value': f}

# Callback para actualizar el valor almacenado de la población inicial
@app.callback(
    Output('store-p0', 'data'),
    [Input('slider-p0', 'value')]
)
def update_store_p0(P0):
    return {'value': P0}

# Callback para actualizar el diagrama de Bifurcación cuando cambia la tasa de fertilidad
@app.callback(
    Output("graph-bifurcation", "figure"),
    [Input('store-f', 'data')]
)
def update_bifurcation_diagram(stored_f):
    f = stored_f['value']
    return bifurcation_diagram(f_min=0, f_max=4, num_points=100)

# Callback para alternar el modal de instrucciones
@app.callback(
    Output("instructions-modal", "is_open"),
    [Input("open-instructions", "n_clicks"), Input("close-instructions", "n_clicks")],
    [State("instructions-modal", "is_open")]
)
def toggle_modal_instructions(open_clicks, close_clicks, is_open):
    if open_clicks or close_clicks:
        return not is_open
    return is_open

# Callback para alternar el modal de detalles
@app.callback(
    Output("details-modal", "is_open"),
    [Input("open-details", "n_clicks"), Input("close-details", "n_clicks")],
    [State("details-modal", "is_open")]
)
def toggle_modal_details(open_clicks, close_clicks, is_open):
    if open_clicks or close_clicks:
        return not is_open
    return is_open

# Callback para alternar el modal de información adicional
@app.callback(
    Output("info-modal", "is_open"),
    [Input("open-info", "n_clicks"), Input("close-info", "n_clicks")],
    [State("info-modal", "is_open")]
)
def toggle_modal_info(open_clicks, close_clicks, is_open):
    if open_clicks or close_clicks:
        return not is_open
    return is_open

# Función callback para actualizar el gráfico
@app.callback(
    Output('matplotlib-plot', 'figure'),
    [Input('store-f', 'data')]  # Puede añadir más inputs si es necesario
)
def update_matplotlib_plot(_):
    return generate_plot()



def ODE_euler(
    *,
    a: float,
    b: float,
    f: Callable[[float, float], float],
    y_t0: float,
    N: int,
) -> tuple[list[float], list[float], float]:
    
    h = (b - a) / N
    t = a
    ts = [t]
    ys = [y_t0]

    for _ in range(N):
        y = ys[-1]
        y += h * f(t, y)
        ys.append(y)

        t += h
        ts.append(t)
    return ys, ts, h

def poblacion_EDO(t, P, f, K):
    return f * P * (1 - P/K)
    
def poblacion_EDO2(t, P, f, K):
    return f * P * (1 - P/K) - P

def generate_plot():
    a = 0          # Tiempo inicial
    b = 4          # Tiempo final
    N = 200       # Número de pasos
    f_values = np.linspace(3, 4, 10)  # Valores de f en el rango [0, 4]
    y0_values = np.arange(0, 1.10, 0.10)  # Valores de y_0 en el rango [0, 1] con saltos de 0.05
    k_values = np.arange(0.10, 1.10, 0.10)
    
    fig, ax = plt.subplots(figsize=(8, 4))

    combinaciones_vistas = set()

    for f_value in f_values:
        for k_value in k_values:
            for y_t0 in y0_values:
                combinacion = (f_value, k_value, y_t0)
                if combinacion not in combinaciones_vistas:
                    combinaciones_vistas.add(combinacion)
                    y_der = lambda t, P: poblacion_EDO(t, P, f_value, k_value)
                    ys, ts, h = ODE_euler(a=a, b=b, f=y_der, y_t0=y_t0, N=N)
                    ax.plot(ts, ys)
    
    for f_value in f_values:
        for k_value in k_values:
            for y_t0 in y0_values:
                combinacion = (f_value, k_value, y_t0)
                if combinacion not in combinaciones_vistas:
                    combinaciones_vistas.add(combinacion)
                    y_der = lambda t, P: poblacion_EDO2(t, P, f_value, k_value)
                    ysa, tsa, ha = ODE_euler(a=a, b=b, f=y_der, y_t0=y_t0, N=N)
                    ax.plot(tsa, ysa)

    ax.set_title("Solución de la EDO de la Población")
    ax.set_xlabel("Tiempo t")
    ax.set_ylabel("P(t)")
    ax.grid(True)

    # Convertir el gráfico Matplotlib a Plotly
    plotly_fig = mpl_to_plotly(fig)
    plt.close(fig)  # Cerrar la figura Matplotlib
    return plotly_fig

# Ejecutar la aplicación Dash
if __name__ == "__main__":
    app.run_server(debug=True)
